In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn import linear_model
from sklearn.utils import shuffle

In [40]:
data = pd.read_csv("student/student-mat.csv", sep=";")

# display the first 10 rows of the data
data.head(10)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10
5,GP,M,16,U,LE3,T,4,3,services,other,...,5,4,2,1,2,5,10,15,15,15
6,GP,M,16,U,LE3,T,2,2,other,other,...,4,4,4,1,1,3,0,12,12,11
7,GP,F,17,U,GT3,A,4,4,other,teacher,...,4,1,4,1,1,1,6,6,5,6
8,GP,M,15,U,LE3,A,3,2,services,other,...,4,2,2,1,1,1,0,16,18,19
9,GP,M,15,U,GT3,T,3,4,other,other,...,5,5,1,1,1,5,0,14,15,15


### Trimming down the data to the ones we are actually going to use in this Linear Regression model


In [41]:
data = data[["G1", "G2", "G3", "studytime", "failures", "absences"]]

# attributes we are going to use here except for the final grade of which we are trying to predict so we will remove it when we are training the model

data.head(10)

# Tried to pick the ones with interger values associated with them just to make my life easier

,G1,G2,G3,studytime,failures,absences
0,5,6,6,2,0,6
1,5,5,6,2,0,4
2,7,8,10,2,3,10
3,15,14,15,3,0,2
4,6,10,10,2,0,4
5,15,15,15,2,0,10
6,12,12,11,2,0,0
7,6,5,6,2,0,6
8,16,18,19,2,0,0
9,14,15,15,2,0,0


### We are going to try to predict "G3" which is Third Grade of a student in the math class 

In [42]:
predict = "G3"  # what we are trying to predict also known as the label

#print(data["G3"])
data["G3"].head()


0     6
1     6
2    10
3    15
4    10
Name: G3, dtype: int64

### Creating the Dependent and Independent Variable

In [43]:
x = np.array(data.drop([predict], axis=1)) # drop method normally drop rows soo the axis parameter is there to specify to drop the column rather
# This is the Independent Variable  WHich is our ATTRIBUTES

y = np.array(data[predict])
# so what we are trying to predict here is the G3 which will be on the Y-axis of Linear regression model (dependent Variable)
#WHICH IS OUR LABELS

In [44]:
print(x)
print(y)

# they all converted to numpy arrays


[[ 5  6  2  0  6]
 [ 5  5  2  0  4]
 [ 7  8  2  3 10]
 ...
 [10  8  1  3  3]
 [11 12  1  0  0]
 [ 8  9  1  0  5]]
[ 6  6 10 15 10 15 11  6 19 15  9 12 14 11 16 14 14 10  5 10 15 15 16 12
  8  8 11 15 11 11 12 17 16 12 15  6 18 15 11 13 11 12 18 11  9  6 11 20
 14  7 13 13 10 11 13 10 15 15  9 16 11 11  9  9 10 15 12  6  8 16 15 10
  5 14 11 10 10 11 10  5 12 11  6 15 10  8  6 14 10  7  8 18  6 10 14 10
 15 10 14  8  5 17 14  6 18 11  8 18 13 16 19 10 13 19  9 16 14 13  8 13
 15 15 13 13  8 12 11  9  0 18  0  0 12 11  0  0  0  0 12 15  0  9 11 13
  0 11  0 11  0 10  0 14 10  0 12  8 13 10 15 12  0  7  0 10  7 12 10 16
  0 14  0 16 10  0  9  9 11  6  9 11  8 12 17  8 12 11 11 15  9 10 13  9
  8 10 14 15 16 10 18 10 16 10 10  6 11  9  7 13 10  7  8 13 14  8 10 15
  4  8  8 10  6  0 17 13 14  7 15 12  9 12 14 11  9 13  6 10 13 12 11  0
 12 12  0 12  0 18 13  8  5 15  8 10  8  8 12  8 13 11 14  0 18  8 12  9
  0 17 10 11 10  0  9 14 11 14 10 12  9  9  8 10  8 10 12 10 11 11 19 12
 14 15 11 

### Converting our X & Y variables into 4 parts (X_Train,Y_Train) & (X_Test, Y_Test)

In [45]:
import sklearn.model_selection


# Splitting them up into 4 different arrays

x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x,y,test_size=0.1)

### Creating our LinearRegression model and the line of best fit

In [46]:
linear = linear_model.LinearRegression()

linear.fit(x_train,y_train)
# fitting the model to the data to find the best fit line

LinearRegression()

### Now Evaluating how good the linear model is doing in terms of Accuracy 

In [47]:
acc = linear.score(x_test,y_test)

print("Accuracy of the model is: ", acc)

Accuracy of the model is:  0.9090688015613676


### Now printing the actual Co-efficient of the line (Y = MX + B) in order to actually use this model to predict new data

In [48]:
print("Coefficient: ", linear.coef_) # this is the slope of the line that is the value of the slope of the line (m)


# so basically a line in 2 dimensional space has 2 coefficients, 
# but this has 5 coefficients because we have 5 attributes that we are using to predict the final grade of the students

print("Intercept: ", linear.intercept_) # this is the y-intercept of the line which is the point where the line crosses the y-axis (B)

Coefficient:  [ 0.13826992  0.99856032 -0.16250488 -0.3230128   0.03947271]
Intercept:  -1.6524951678572855


In [ ]:
predictions = linear.predict(x_test)

for i in range(len(predictions)):
    print(predictions[i], x_test[i], y_test[i])

# This loop iterates over the range of the length of the predictions array. For each iteration, it prints the predicted value, the corresponding feature set from x_test, and the actual target value from y_test.

# The predicted value is the model’s guess for the final grade of the student. The feature set is the set of attributes that the model used to make the prediction. The actual target value is the real final grade of the student.
 
# KEY : 9.272148547861....(Predicted Value) [G1, G2, studytime, failures, absences] YYY (Actual Value)

9.272148547861493 [ 8 10  2  0  4] 9
-1.126796295175815 [12  0  3  2  0] 0
9.548688396027892 [10 10  2  0  4] 11
5.817801941873434 [7 7 3 0 0] 8
9.276762587535925 [ 8 10  1  0  0] 11
16.21616220131907 [14 16  1  0  3] 16
9.548688396027892 [10 10  2  0  4] 10
6.2343777827136915 [8 7 1 1 7] 8
11.617985109001623 [14 12  2  1  0] 12
14.93667887584219 [14 15  2  0  0] 15
8.308677779410777 [10  9  3  0  2] 10
9.746282826474769 [12 10  2  0  2] 11
16.187313210614352 [15 16  3  0  7] 15
10.219955332791914 [12 10  2  0 14] 11
18.52029821271265 [18 18  3  0  5] 19
3.8645371566913456 [5 5 2 0 4] 6
10.867644430364532 [10 11  1  0  8] 10
7.950908876731729 [10  8  1  0 10] 9
4.552936244029048 [8 6 2 2 2] 5
10.665897732539353 [12 11  2  0  0] 12
13.874180006019973 [13 14  3  0  6] 14
13.417844774265552 [14 13  1  0  8] 13
9.119980708848658 [11  9  1  1 19] 10
7.61089225482335 [11  8  2  0  2] 8
14.12648490315609 [16 14  4  0  6] 15
15.21760187753496 [14 15  1  0  3] 16
6.723288046860317 [ 9  7  2  1 

## A simple Linear Regression Model